##### Load scraped data from csv file

In [18]:
import pandas as pd
import scraper
import importlib

importlib.reload(scraper) 

<module 'scraper' from 'c:\\Users\\USER\\Documents\\HouseSigma Scraper\\housesigma-scraper\\scraper.py'>

In [19]:
# Load the DataFrame from the CSV file
input_df = pd.read_csv('filtered_sold_data.csv')
input_df = input_df.drop_duplicates().reset_index(drop=True)

print(input_df.shape)
print(input_df.columns)
print("\n ===================== \n")
print(input_df.dtypes)
print("\n ===================== \n")
print(input_df.head())

(764, 10)
Index(['Asking Price', 'Sold Price', 'Address', 'Unit Type', 'Bedrooms',
       'Bathrooms', 'Postal Code', 'Listing URL', 'Bedroom Category',
       'Postal Code Area'],
      dtype='object')


Asking Price        float64
Sold Price          float64
Address              object
Unit Type            object
Bedrooms             object
Bathrooms             int64
Postal Code          object
Listing URL          object
Bedroom Category     object
Postal Code Area     object
dtype: object


   Asking Price  Sold Price  \
0      998000.0    975000.0   
1     1259000.0   1349000.0   
2     3599900.0   3250000.0   
3      825000.0    780000.0   
4      559000.0    529000.0   

                                             Address        Unit Type  \
0   623 - 88 Colgate Ave , Toronto - South Riverdale        Condo Apt   
1  84A Aylesworth Ave , Scarborough - Birchcliffe...         Detached   
2            501 Avonwood Dr , Mississauga - Mineola         Detached   
3  314 - 199 Pine Gr

##### Extract maintenance fees from urls

In [20]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

print("🔄 Reinitializing WebDriver...")

options = webdriver.ChromeOptions()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# ✅ Open HouseSigma for manual login
driver.get("https://housesigma.com")
input("🔑 Press Enter after logging in manually...")  # ⬅️ Login manually, then press Enter

input_extracted_data = scraper.process_with_progress(input_df, driver, progress_interval=100)

# ✅ Convert the extracted dictionary output into a DataFrame
input_extracted_df = input_extracted_data.apply(pd.Series)

# ✅ Close the browser after processing all listings
driver.quit()
print(input_extracted_df.shape)
print("\n✅ All listings processed. Browser closed.")

🔄 Reinitializing WebDriver...
🚀 Processing row 1/764
🚀 Processing row 101/764
🚀 Processing row 201/764
🚀 Processing row 301/764
🚀 Processing row 401/764
🚀 Processing row 501/764
🚀 Processing row 601/764
🚀 Processing row 701/764
🚀 Processing row 764/764
(764, 3)

✅ All listings processed. Browser closed.


In [21]:
# Save the DataFrame to a CSV file
input_extracted_df.to_csv('input_extracted_data.csv', index=False)

# Print a confirmation message
print("DataFrame has been saved to 'input_extracted_data.csv'")

DataFrame has been saved to 'input_extracted_data.csv'


In [26]:
# Load the DataFrame from the CSV file
input_extracted_df = pd.read_csv('input_extracted_data.csv')

print(input_extracted_df.shape)
print("\n ===================== \n")

# Extract numeric values from the 'Maintenance Fees' column
input_extracted_df['Maintenance Fees'] = (
    input_extracted_df['Maintenance Fees']
    .str.replace(r'[^\d.]', '', regex=True)  # Remove non-numeric characters
    .replace('', '0')  # Replace empty strings with '0'
    .astype(float)  # Convert to float
    .fillna(0)  # Replace NaN with 0
)

# Print the first few rows to verify
print(input_extracted_df.head())

(764, 3)


  Sold Days Ago  Maintenance Fees  \
0             5             708.0   
1            11               0.0   
2            11               0.0   
3            11             414.0   
4            12             728.0   

                                    Unit Description  
0  Inviting you to Suite 623 at the exceptional S...  
1  Luxury, detached custom-built 4 bed + 5 bths +...  
2  Where luxury is sprinkled with a traditional m...  
3  Don't Miss This Exceptional Opportunity To Own...  
4  Welcome to Tridel-Built 'Qwest' at Prime Locat...  


##### Merge input data and extracted data (sold days ago values, maintenance fees, unit description)

In [27]:
# Concatenate the DataFrames along the columns axis
merged_df = pd.concat([input_df, input_extracted_df], axis=1)

# Print the first few rows to verify
print(merged_df.shape)
print("\n ===================== \n")
print(merged_df.head())

# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_data.csv', index=False)

# Print a confirmation message
print("Merged DataFrame has been saved to 'merged_data.csv'")

(764, 13)


   Asking Price  Sold Price  \
0      998000.0    975000.0   
1     1259000.0   1349000.0   
2     3599900.0   3250000.0   
3      825000.0    780000.0   
4      559000.0    529000.0   

                                             Address        Unit Type  \
0   623 - 88 Colgate Ave , Toronto - South Riverdale        Condo Apt   
1  84A Aylesworth Ave , Scarborough - Birchcliffe...         Detached   
2            501 Avonwood Dr , Mississauga - Mineola         Detached   
3  314 - 199 Pine Grove Rd , Vaughan - East Woodb...  Condo Townhouse   
4  517 - 168 Simcoe St , Toronto - Waterfront Com...        Condo Apt   

  Bedrooms  Bathrooms Postal Code  \
0        2          2      M4M0A6   
1        4          5      M1N2J6   
2        4          4      L5G1Y8   
3        2          2      L4L0H8   
4        1          1      M5H4C9   

                                         Listing URL Bedroom Category  \
0  https://housesigma.com/on/toronto-real-estate/...              

##### Check maintenance fees values for condo units

In [29]:
# Filter the DataFrame for rows where 'Unit Type' contains 'Condo' (case-insensitive) and 'Maintenance Fees' are 0
condo_zero_fees = merged_df[(merged_df['Unit Type'].str.contains('condo', case=False, na=False)) & (merged_df['Maintenance Fees'] == 0)]

# Print the number of rows that match the criteria
print(f"Number of rows where 'Unit Type' contains 'condo' and 'Maintenance Fees' are 0: {condo_zero_fees.shape[0]}")

# Optionally, print the first few rows to verify
pd.set_option('display.max_colwidth', None)
print(condo_zero_fees.head())

Number of rows where 'Unit Type' contains 'condo' and 'Maintenance Fees' are 0: 3
     Asking Price  Sold Price  \
315      698000.0    680000.0   
509      695000.0    640000.0   
590      599000.0    525000.0   

                                                         Address  Unit Type  \
315       608 - 8 Eglinton Ave E , Toronto - Mount Pleasant West  Condo Apt   
509  809 - 77 Harbour Sq W , Toronto - Waterfront Communities C1  Condo Apt   
590     3007 - 50 Charles St E , Toronto - Church-Yonge Corridor  Condo Apt   

    Bedrooms  Bathrooms Postal Code  \
315        2          2      M4P0C1   
509        1          1      M5J2S2   
590        1          1      M4Y0C3   

                                                                                   Listing URL  \
315  https://housesigma.com/on/toronto-real-estate/608-8-eglinton-ave-e/home/damgL7AqBJV7Z1MW/   
509   https://housesigma.com/on/toronto-real-estate/809-77-harbour-sq-w/home/weQp5yOoorvyd0ZE/   
590  https://hous